# unified_with_date

In [1]:
from nltk.tokenize import MWETokenizer, TweetTokenizer, WordPunctTokenizer 
from nltk.tokenize import word_tokenize
import re

VALUES = ['>>>=', '>>=', '<<=',  '%=', '^=', '|=', '&=', '/=',
                  '*=', '-=', '+=', '<<', '--', '++', '||', '&&', '!=',
                  '>=', '<=', '==', '%', '^', '|', '&', '/', '*', '-',
                  '+', ':', '?', '~', '!', '<', '>', '=', '...', '->', '::', '\\', '\\\\', '\*', '*\\', '\\\\\\']
INFIX = ['||', '&&', '|', '^', '&', '==', '!=', '<', '>', '<=', '>=', 
             '<<', '>>', '>>>', '+', '-', '*', '/', '%']
PREFIX = ['++', '--', '!', '~', '+', '-']
POSTFIX = ['++', '--']
ASSIGNMENT = ['=', '+=', '-=', '*=', '/=', '&=', '|=', '^=', '%=', '<<=', '>>=', '>>>=']
LAMBDA = ['->']
COMMENT = ['//', '/*', '*/']
METHOD_REFERENCE = ['::']

WHITESPACE_DICT = {"                ":'<|16-s|>', 
                   "            ":'<|12-s|>', 
                   "        ":'<|8-s|>', 
                   "    ":'<|4-s|>', 
                   "  ":'<|2-s|>', " ":'<|s|>',
                   "\t\t\t\t":'<|4-t|>',"\t\t\t":'<|3-t|>',"\t\t":'<|2-t|>',"\t":'<|t|>',"\n":'<|nl|>'}

WHITESPACES = [WHITESPACE_DICT[x] for x in WHITESPACE_DICT]

REVERSE_WHITESPACE_DICT = {}
for key in WHITESPACE_DICT:
    value = WHITESPACE_DICT[key]
    REVERSE_WHITESPACE_DICT[value] = key

CUSTOM_TOKEN = ["<|startcode|>", "<|endcode|>", "<|startfocus|>", "<|endfocus|>", "<|startcomment|>", "<|endcomment|>", 
                   "<|stringliteral|>", "<|singlelinecomment|>", "<|multilinecomment|>", "<|del|>"]

values = list(set(INFIX+PREFIX+POSTFIX+ASSIGNMENT+LAMBDA+COMMENT+METHOD_REFERENCE))
token_phrases = []
word_punct_tokenizer = WordPunctTokenizer()
for tok in CUSTOM_TOKEN+WHITESPACES:
    temp = tuple(word_punct_tokenizer.tokenize(tok))
    token_phrases.append(temp)
for w in values:
    temp = tuple(MWETokenizer().tokenize(w))
    if len(temp) > 1:
        token_phrases.append(temp)

word_punct_tokenizer = WordPunctTokenizer()
tweet_tokenizer = TweetTokenizer()
mwe_tokenizer = MWETokenizer(token_phrases, separator="")

def state(c):
    n = ord(c)
    if n>=97 and n<=122: # lower case
        return 1
    elif n>=65 and n<=90: # upper case
        return 2
    elif n>=48 and n<=57: # numbers
        return 3
    elif c.isspace(): # whitespaces
        return 4
    elif c in ['_', '$']: 
        return 5
    elif n < 128:
        return 6
    else:
        return 7

def space_up(s):
    if s is None or s == "":
        return ""
    new_s = s[0]
    for i in range(1,len(s)):
        prev_state = state(s[i-1])
        curr_state = state(s[i])
        if prev_state in [1,2] and curr_state in [3]:
            new_s += " "
        elif prev_state in [1] and curr_state in [2]:
            new_s += " "
        elif prev_state in [3] and curr_state in [1,2]:
            new_s += " "
        elif prev_state in [1,2,3] and curr_state in [5]:
            new_s += " "
        elif prev_state in [5] and curr_state in [1,2,3]:
            new_s += " "
        new_s+=s[i]
    return new_s

def white_space_tokenize(s):
    for x in WHITESPACE_DICT:
        s = s.replace(x, WHITESPACE_DICT[x])
    for key in REVERSE_WHITESPACE_DICT:
        #val = REVERSE_WHITESPACE_DICT[x]
        s = s.replace(key, " "+key+" ")
    return s

def extreme_tokenization(comment):
    comment = white_space_tokenize(comment)
    comment = space_up(comment)
    tokenized = tweet_tokenizer.tokenize(comment)
    tokenized = word_punct_tokenizer.tokenize(' '.join(tokenized))
    tokenized = mwe_tokenizer.tokenize(tokenized)
    tokenized_comment = ' '.join(tokenized)
    tokenized_comment = re.sub(r'[^\x00-\x7f]',r'', tokenized_comment)
    tokenized = tokenized_comment.split()
    return tokenized

def extreme_detokenization(tokens):
    s = ""
    for token in tokens:
        if token in REVERSE_WHITESPACE_DICT:
            s+= REVERSE_WHITESPACE_DICT[token]
        else:
            s+= token
    return s

In [108]:
import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
def read(path):
    return open(path, 'r', encoding = 'utf-8').read()


BASE="C:/research_stuff/codes/"
unique_data = getJsonData('E:/APR/DATA/unique_data_processed_with_Date_multithread_idx_400_feb_03.json')
#print(unique_data[0])

{'status': 'update', 'message': 'What\'s "us" ?', 'comment_id': 'f98e907f_b5dc8856', 'target': '     * <BR>Then the test sets a breakpoint so that the tested thread (DebuggeeThread) gets suspended\n     * by an event. Once this thread is suspended, we send it an ObjectReference.InvokeMethod\n     * command to initiate a method invocation executing in that thread. The method will synchronize\n     * with the test, waiting for a signal to continue its execution.\n     * <BR>While the tested thread waits for the signal, we send a VirtualMachine.Dispose command to\n     * the debuggee and sends the expected signal so the tested thread completes the method\n     * invocation.\n', 'code_snippet': "     * This testcase exercises VirtualMachine.Dispose command when a thread, suspended by an\n     * event, is still invoking a method.\n     * <BR>At first the test starts DisposeDuringInvokeDebuggee debuggee.\n<|startfocus|>\n     * <BR>Then the test sets a breakpoint a tested thread (DebuggeeThr

In [109]:
from os import path
import json
import re
import subprocess
import nltk
from pprint import pprint
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm
nltk.download('punkt')

def get_source_target(file1, file2, line_number, change_window_size=5):
    source_target = subprocess.check_output(['java', '-jar', 'ChangedLineF.jar', \
                                file1, file2, str(line_number), str(change_window_size)]).decode("utf-8")
    (source, target) = source_target.split("<|source-target-sep|>")
    return source, target

def end_scope(start_index, lines):
    #print(lines)
    counter = 0
    end_index = -1
    found = False
    for index in range(start_index, len(lines), 1):
        #print(lines[index])
        #print(counter)
        for char in lines[index]:
            if(char=='{'):
                counter+=1
                found = True
            elif(char == "}"):
                counter-=1
                found = True
            if(counter==0 and found):
                end_index = index
                break
        if end_index!=-1:
            break
    return end_index

unique_data_processed = []
MAX_NUM_TOKEN = 800
UP_TOKEN = MAX_NUM_TOKEN//2
c = 0
ec = 0
errors = []
tufano_less_then_50 = []
tufano_less_then_100 = []
vis_idx = []
#for idx, data in enumerate(unique_data):#[2428:2429]#[1000:1001]
for idx, data in enumerate(unique_data): #1:75
    #if(idx>3000):
    #    break
    if(idx%100==0):
        print(idx, len(tufano_less_then_50), len(tufano_less_then_100))
    #if len(tufano_less_then_50) >=10 and len(tufano_less_then_100)>=10:
    #    break
    if data['project_name']!='android_':
        continue
    code_file_name = data['code_file_name']
    base_patch_number = data['base_patch_number']
    changed_patch_number = data['changed_patch_number']
    main_code_path = BASE+code_file_name+'/'+str(base_patch_number)+'.java'
    changed_code_path = BASE+code_file_name+'/'+str(changed_patch_number)+'.java'
    line_no = data['base_code_line_number']
    main_code = read(main_code_path)
    changed_code = read(changed_code_path)
    sample = main_code
    message = data['message']
    comment_id = data['comment_id']
    #print("line no = ", line_no)
    sp_start_func= -1
    sp_end_func= len(sample.split('\n'))   
    funcs = re.findall(r"(public|protected|private|static|\s) +([\w\<\>\[\]]+\s+(\w+)) *\([^\)]*\) *(\{?|[^;])",sample)
    func_scopes = []
    lines = sample.split("\n")
    #print(funcs)
    #continue
    visited_func = set()
    for func in funcs:
        #print(func)
        if(func[-1]!= "{"):
            continue
        fc = list(func)
        fc = fc[-3]
        s = "".join(fc)
        s = s.strip()
        found = False
        for index in range(0, len(lines), 1):
            if s in lines[index]:
                start_func = index
                end_func = end_scope(index, lines)
                func_scopes.append((start_func,end_func))
                if s in visited_func:
                    continue
                if s not in visited_func:
                    visited_func.add(s)
                    #func_list.append(lines[index].replace("{", "").strip())
                if(start_func<=line_no<=end_func):
                    found = True
                    special_func = "\n".join(lines[start_func:end_func+1])
                    sp_start_func= start_func
                    sp_end_func= end_func
                    #print("comes")
                    if s not in extreme_detokenization(data['tokenized_code_snippet']):
                        continue
                    last_char =  extreme_detokenization(data['tokenized_code_snippet']).strip()[-1]
                    if last_char!='}':
                        continue
                    detokenized_code = extreme_detokenization(data['tokenized_code_snippet'])
                    #print(detokenized_code)
                    start_f = detokenized_code.find("<|startfocus|>\n")
                    end_f = detokenized_code.find("<|endfocus|>\n")
                    #print("se ", start_f, " ", end_f)
                    repl_st = detokenized_code[start_f:end_f+14]
                    #print(repl_st)
                    #print(data['target'])
                    with open("temp\\temp.java", 'w', encoding='utf-8') as f:
                        f.write(extreme_detokenization(data['tokenized_code_snippet']).replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", ""))
                    with open("temp\\temp2.java", 'w', encoding='utf-8') as f:
                        f.write(extreme_detokenization(data['tokenized_code_snippet']).replace(repl_st, data['target'].replace("\n<|del|>\n", "")))#replace("\n<|del|>\n", ""))
                    
                    java_cmd = "java -jar src2abs-0.1-jar-with-dependencies.jar pair method "
                    java_cmd = java_cmd + "temp\\temp.java temp\\temp2.java"+" "
                    java_cmd = java_cmd+"temp\\processed.java temp\\processed2.java"+' idioms.csv'
                    check = subprocess.call(java_cmd.replace('/', '\\'), shell=True)
                    #print(java_cmd)
                    if check != 0:
                        print(java_cmd)
                        print("java cmd execusion failed on file ", f)
                        continue
                    '''
                    check = subprocess.call(java_cmd.replace('/', '\\').replace("temp.java", "temp2.java").replace("processed", "processed2"), shell=True)
                    #print(java_cmd)
                    if check != 0:
                        print(java_cmd)
                        print("java cmd execusion failed on file ", f)
                        continue
                    #print("comes")
                    '''
                    file_data = read("temp\\processed.java")
                    file_data_y = read("temp\\processed2.java")
                    #print(file_data)
                    #print(idx," ",  len(file_data.split(" "))," ", len(file_data_y.split(" ")) )
                    if(len(file_data.split(" "))<=50 and len(file_data_y.split(" "))<=50):
                        dt = {}
                        dt['idx'] = idx
                        dt['status'] = data['status']
                        dt['global_index'] = data['global_index']
                        dt['project_name'] = data['project_name']
                        dt['written_on'] = data['written_on']
                        #dt['source'] = file_data
                        dt['tufano_target'] = file_data_y
                        dt['comment'] = data['message']
                        dt['tokenized_comment'] = data['tokenized_comment']
                        dt['tufano_code'] = file_data
                        dt['our_target'] = data['target']
                        dt['our_code'] = data['code_snippet'] #.replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", "")
                        dt['tokenized_code'] = data['tokenized_code_snippet']
                        dt['written_on'] = data['written_on']
                        
                        dt['base_code_line_number'] = data['base_code_line_number']
                        dt['base_patch_number'] = data['base_patch_number']
                        dt['changed_patch_number'] = data['changed_patch_number']
                        dt['code_file_name'] = data['code_file_name']
                        dt['comment_id'] = data['comment_id']
                        dt['class_list'] = data['class_list']
                        dt['func_list'] = data['func_list']
                        dt['prime_var_dic'] = data['prime_var_dic']
                        #print("50->", idx)
                        tufano_less_then_50.append(dt)
                        #func_less_then_50.append(file_data)
                        break
                    if(len(file_data.split(" "))<=100 and len(file_data_y.split(" "))<=100):
                        dt = {}
                        dt['idx'] = idx
                        dt['status'] = data['status']
                        dt['global_index'] = data['global_index']
                        dt['project_name'] = data['project_name']
                        dt['written_on'] = data['written_on']
                        #dt['source'] = file_data
                        dt['tufano_target'] = file_data_y
                        dt['comment'] = data['message']
                        dt['tokenized_comment'] = data['tokenized_comment']
                        dt['tufano_code'] = file_data
                        dt['our_target'] = data['target']
                        dt['our_code'] = data['code_snippet'] #.replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", "")
                        dt['tokenized_code'] = data['tokenized_code_snippet']
                        dt['written_on'] = data['written_on']
                        
                        dt['base_code_line_number'] = data['base_code_line_number']
                        dt['base_patch_number'] = data['base_patch_number']
                        dt['changed_patch_number'] = data['changed_patch_number']
                        dt['code_file_name'] = data['code_file_name']
                        dt['comment_id'] = data['comment_id']
                        dt['class_list'] = data['class_list']
                        dt['func_list'] = data['func_list']
                        dt['prime_var_dic'] = data['prime_var_dic']
                        tufano_less_then_100.append(dt)
                        break
                    

0 0 0


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


100 10 7
200 17 11
300 22 14
400 31 19
500 34 21
600 38 23
700 38 28
800 40 29
900 40 33
1000 43 37
1100 43 38
1200 44 42
1300 46 44
1400 50 45
1500 52 47
1600 54 48
1700 56 53
1800 57 54
1900 57 55
2000 57 56
2100 60 56
2200 60 56
2300 60 57
2400 61 59
2500 61 60
2600 61 62
2700 61 63
2800 62 64
2900 64 66
3000 64 69
3100 65 70
3200 67 71
3300 68 75
3400 68 76
3500 68 77
3600 68 78
3700 70 79
3800 71 82
3900 72 82
4000 74 82
4100 76 82
4200 76 82
4300 78 82
4400 79 83
4500 79 84
4600 81 85
4700 82 86
4800 86 88
4900 87 88
5000 88 90
5100 88 97
5200 89 99
5300 90 101
5400 91 102
5500 93 104
5600 95 106
5700 96 109
5800 96 111
5900 96 114
6000 96 115
6100 99 116
6200 101 116
6300 104 117
6400 107 117
6500 108 119
6600 110 119
6700 110 122
6800 112 123
6900 113 124
7000 113 125
7100 114 126
7200 117 126
7300 119 129
7400 122 132
7500 123 134
7600 126 137
7700 128 137
7800 131 138
7900 131 140
8000 131 143
8100 132 143
8200 133 144
8300 133 145
8400 133 146
8500 135 146
8600 136 146
8700 

In [110]:
print(len(tufano_less_then_50))
print(len(tufano_less_then_100))
with open('less_then_50_on_400_v2.json', 'w', encoding="utf8") as f:  # writing JSON object
    json.dump(tufano_less_then_50, f)
with open('less_then_100_on_400_v2.json', 'w', encoding="utf8") as f:  # writing JSON object
    json.dump(tufano_less_then_100, f)

1275
1628


In [142]:
pprint(tufano_less_then_50)

[{'base_code_line_number': 71,
  'base_patch_number': 11,
  'changed_patch_number': 12,
  'class_list': ['NewCloudModuleDynamicWizardPath',
                 'represents',
                 'NewCloudModuleTemplate'],
  'code_file_name': 'android_4548',
  'comment': 'missing space before the new.',
  'comment_id': 'd06988a3_4c321266',
  'func_list': ['protected void init()',
                'public String getPathName()',
                'public Iterable<ModuleTemplate> getModuleTemplates()',
                'public boolean isPathVisible()',
                'public boolean performFinishingActions()',
                'public NewCloudModuleTemplate(@NotNull List<TemplateInfo> '
                'templates)',
                'public String getName()',
                'public String getDescription()',
                'public Icon getIcon()',
                'public void updateWizardStateOnSelection(ScopedStateStore '
                'state)',
                'public boolean isGalleryModuleType(

                   'instanceof TYPE_2 ) ) { return null ; } TYPE_2 VAR_2 = ( '
                   'TYPE_2 ) VAR_1 ; try { TYPE_3 VAR_3 = new TYPE_3 ( ) ; '
                   'TYPE_4 VAR_4 = new TYPE_4 ( VAR_3 ) ; VAR_4 . METHOD_2 ( '
                   'VAR_5 ) ; byte [ ] data = VAR_2 . METHOD_3 ( ) ; VAR_4 . '
                   'METHOD_2 ( data . length ) ; VAR_4 . METHOD_4 ( data ) ; '
                   'TYPE_5 [ ] VAR_6 = VAR_1 . METHOD_5 ( ) ; VAR_4 . METHOD_2 '
                   '( VAR_6 . length ) ; for ( TYPE_5 VAR_7 : VAR_6 ) { data = '
                   'VAR_7 . METHOD_3 ( ) ; VAR_4 . METHOD_2 ( data . length ) '
                   '; VAR_4 . METHOD_4 ( data ) ; } TYPE_6 < byte [ ] > VAR_8 '
                   '= VAR_2 . METHOD_6 ( ) ; VAR_4 . METHOD_2 ( VAR_8 . size ( '
                   ') ) ; for ( byte [ ] VAR_9 : VAR_8 ) { VAR_4 . METHOD_2 ( '
                   'VAR_9 . length ) ; VAR_4 . METHOD_4 ( VAR_9 ) ; } byte [ ] '
                   'VAR_10 = VAR_2 . METHOD

                'public List<Camera> getCameras()',
                'public void addCamera(@NonNull Camera c)',
                'public void addAllCameras(@NonNull Collection<Camera> cs)',
                'public Camera getCamera(int i)',
                'public Keyboard getKeyboard()',
                'public void setKeyboard(@NonNull Keyboard keyboard)',
                'public Navigation getNav()',
                'public void setNav(@NonNull Navigation n)',
                'public Storage getRam()',
                'public void setRam(@NonNull Storage ram)',
                'public ButtonType getButtonType()',
                'public void setButtonType(@NonNull ButtonType bt)',
                'public List<Storage> getInternalStorage()',
                'public void addInternalStorage(@NonNull Storage is)',
                'public void addAllInternalStorage(@NonNull '
                'Collection<Storage> is)',
                'public List<Storage> getRemovableStorage()',
          

  'code_file_name': 'android_3767',
  'comment': 'Why not keep location, moreover location is homogeneous with '
             'others exceptions.',
  'comment_id': '8e07f5ca_97cd44ca',
  'func_list': ['public NotFileOrDirectoryException(@Nonnull FileOrDirLocation '
                'expectedLocation)'],
  'global_index': 3005,
  'idx': 88,
  'our_code': '<|startfocus|>\n'
              '  public NotFileOrDirectoryException(@Nonnull FileOrDirLocation '
              'expectedLocation) {\n'
              '    this(expectedLocation, null);\n'
              '<|endfocus|>\n'
              '  }',
  'our_target': '  public NotFileOrDirectoryException(@Nonnull '
                'FileOrDirLocation location) {\n'
                '    this(location, null);\n',
  'prime_var_dic': {'long': 1},
  'project_name': 'android_',
  'status': 'update',
  'tokenized_code': ['<|startfocus|>',
                     '<|nl|>',
                     '<|2-s|>',
                     'public',
                     '<|

                        '<|s|>',
                        'with',
                        '<|s|>',
                        'the',
                        '<|s|>',
                        'string',
                        '<|s|>',
                        '"',
                        'Exception',
                        '"',
                        '<|s|>',
                        'and',
                        '<|s|>',
                        'replace',
                        '<|s|>',
                        'it',
                        '<|s|>',
                        'with',
                        '<|s|>',
                        'for',
                        '<|s|>',
                        'example',
                        '<|s|>',
                        '"',
                        'E',
                        '*',
                        '"',
                        '.'],
  'tufano_code': 'private static TYPE_1 METHOD_1 ( TYPE_1 VAR_1 ) { int VAR_2 '
                 '= 0 ; w

...

  'comment': 'would be cool to maybe show the module locations when hovering '
             'over modules here?',
  'comment_id': '59a69bd4_f950b79f',
  'func_list': ['public static AndroidProjectStructureConfigurable '
                'getInstance(@NotNull Project project)',
                'public boolean showDialogAndSelect(@NotNull final Module '
                'module)',
                'return doShowDialog(new Runnable()',
                'public void run()',
                'public boolean showDialogAndSelect(@NotNull final Module '
                'module)',
                'private boolean doShowDialog(@Nullable Runnable advanceInit)',
                'public AndroidProjectStructureConfigurable(@NotNull Project '
                'project)',
                'public void dispose()',
                'public String getDisplayName()',
                'public String getHelpTopic()',
                'public JComponent createComponent()',
                'public Color getBackground()

                'public boolean isDevice(String id)',
                'public ConfigurationToolbarFixture '
                'requireOrientation(@NotNull String name)',
                'public ConfigurationToolbarFixture requireTheme(@NotNull '
                'String theme)',
                'public void toggleOrientation()',
                'public void createLandscapeVariation()',
                'public void chooseLocale(@NotNull String locale)',
                'public void removePreviews()',
                'public void chooseDevice(String labelPrefix)',
                'public void createOtherVariation(@NotNull String variation)',
                'private Configuration getConfiguration()',
                'private JButton findToolbarButton(@NotNull final String '
                'tooltip)',
                'protected boolean isMatching(JButton button)',
                'private void clickPopupMenuItem(@NotNull String label)'],
  'global_index': 12649,
  'idx': 78959,
  'our_code'

                   'METHOD_4 ( VAR_11 . length ) ; VAR_6 . METHOD_6 ( VAR_11 ) '
                   '; } } return VAR_5 . METHOD_10 ( ) ; } catch ( TYPE_8 '
                   'VAR_13 ) { VAR_14 . METHOD_11 ( STRING_2 + VAR_13 . '
                   'METHOD_12 ( ) ) ; return null ; } catch ( TYPE_9 VAR_13 ) '
                   '{ METHOD_13 ( VAR_13 ) ; return null ; } }',
  'written_on': '2013-12-30 23:23:43'},
 {'base_code_line_number': 46,
  'base_patch_number': 2,
  'changed_patch_number': 3,
  'class_list': ['JAbstractMethodBody'],
  'code_file_name': 'android_2285',
  'comment': 'Why pass parent here? Remove parent, or make the method static. '
             'Another question is we add a isValidParent, or a more general '
             'checkNodeValidity, and rename the ParentChecker as a ASTChecker?',
  'comment_id': '50baf7ee_5e85454a',
  'func_list': ['protected JAbstractMethodBody(SourceInfo info)',
                'public JMethod getMethod()',
                'public void setM

                     '<|s|>',
                     'must',
                     '<|s|>',
                     'have',
                     '<|s|>',
                     'taken',
                     '<|s|>',
                     'a',
                     '<|s|>',
                     'full',
                     '<|s|>',
                     'timeout',
                     '<|s|>',
                     'interval',
                     '<|s|>',
                     'for',
                     '<|s|>',
                     'the',
                     '<|s|>',
                     'finalizer',
                     '<|nl|>',
                     '<|12-s|>',
                     '//',
                     '<|s|>',
                     'daemon',
                     '<|s|>',
                     'to',
                     '<|s|>',
                     'get',
                     '<|s|>',
                     'get',
                     '<|s|>',
                     'from',
                  

                     ')',
                     ')',
                     '<|s|>',
                     '{',
                     '<|nl|>',
                     '<|12-s|>',
                     'System',
                     '.',
                     'out',
                     '.',
                     'println',
                     '(',
                     '"',
                     'PREDEX',
                     '<|s|>',
                     'CACHE',
                     '<|s|>',
                     'HITS',
                     ':',
                     '<|2-s|>',
                     '<|s|>',
                     '"',
                     '<|s|>',
                     '+',
                     '<|s|>',
                     'm',
                     'Hits',
                     ');',
                     '<|nl|>',
                     '<|12-s|>',
                     'System',
                     '.',
                     'out',
                     '.',
                     'prin

In [14]:
tufano_less_then_50 = getJsonData("E:/APR/DATA/less_then_50_on_400.json")
tufano_less_then_100 = getJsonData("E:/APR/DATA/less_then_100_on_400.json")

# MAIN dataset duplicate removal

In [194]:
import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
def read(path):
    return open(path, 'r', encoding = 'utf-8').read()


BASE="C:/research_stuff/codes/"
unique_data = getJsonData('unique_data_processed_with_Date_multithread_idx_400_feb_06_v2.json')
print(len(unique_data))

95352


In [195]:
import traceback
existing_ids = set()
duplicate_ids = set()
unique_dat = []
i = 0
target_size = 400
error_count = 0
error_list = []

for x in unique_data:
    i+= 1
    try:
        focuslen = x['tokenized_code_snippet'].index('<|endfocus|>') - x['tokenized_code_snippet'].index('<|startfocus|>')
        key = merge_code_comment(x)
        if key not in existing_ids and len(x['tokenized_target']) <= target_size and focuslen <= target_size:
            existing_ids.add(key)
            unique_dat.append(x)
        elif key not in duplicate_ids:
            duplicate_ids.add(key)
    except Exception as e:
        print(e)
        x['error'] = e
        error_list.append(x)
        error_count += 1
print(len(unique_dat))

85440


In [196]:
import json
with open('unique_data_processed_with_multithread_v2.json', 'w', encoding="utf8") as f:  # writing JSON object
    json.dump(unique_dat, f)

# Remove Duplicates

In [15]:
from os import path
import json
import re
import subprocess
import nltk
from pprint import pprint
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm
nltk.download('punkt')

def merge_code_comment(x):
    message = x['message']
    source = ' '.join(x['tokenized_code_snippet'])
    target = ' '.join(x['target'])
    merged = message + '<>' + source + '<>' + target
    return merged

def get_source_target(file1, file2, line_number, change_window_size=5):
    source_target = subprocess.check_output(['java', '-jar', 'ChangedLineF.jar', \
                                file1, file2, str(line_number), str(change_window_size)]).decode("utf-8")
    (source, target) = source_target.split("<|source-target-sep|>")
    return source, target

def end_scope(start_index, lines):
    #print(lines)
    counter = 0
    end_index = -1
    found = False
    for index in range(start_index, len(lines), 1):
        #print(lines[index])
        #print(counter)
        for char in lines[index]:
            if(char=='{'):
                counter+=1
                found = True
            elif(char == "}"):
                counter-=1
                found = True
            if(counter==0 and found):
                end_index = index
                break
        if end_index!=-1:
            break
    return end_index

import traceback
existing_ids = set()
duplicate_ids = set()
unique_dat = []
i = 0
target_size = 400
error_count = 0
error_list = []

for dt in tufano_less_then_50:
    #print(dt['idx'])
    x = unique_data[dt['idx']]
    i+= 1
    try:
        focuslen = x['tokenized_code_snippet'].index('<|endfocus|>') - x['tokenized_code_snippet'].index('<|startfocus|>')
        key = merge_code_comment(x)
        if key not in existing_ids and len(x['tokenized_target']) <= target_size and focuslen <= target_size:
            existing_ids.add(key)
            unique_dat.append(i-1)
        elif key not in duplicate_ids:
            duplicate_ids.add(key)
    except Exception as e:
        print(e)
        x['error'] = e
        error_list.append(x)
        error_count += 1
        #print(x)
        #break
print(len(tufano_less_then_50))

manual_less_then_50 = []

for i in unique_dat: #1:75
    dt = tufano_less_then_50[i]
    '''
    #print(dt['idx'])
    #print(dt['source'])
    data = unique_data[dt['idx']]
    code_file_name = data['code_file_name']
    base_patch_number = data['base_patch_number']
    changed_patch_number = data['changed_patch_number']
    main_code_path = BASE+code_file_name+'/'+str(base_patch_number)+'.java'
    changed_code_path = BASE+code_file_name+'/'+str(changed_patch_number)+'.java'
    line_no = data['base_code_line_number']
    main_code = read(main_code_path)
    changed_code = read(changed_code_path)
    sample = main_code
    message = data['message']
    comment_id = data['comment_id']
    detokenized_code = extreme_detokenization(data['tokenized_code_snippet'])
    #print(detokenized_code.replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", ""))
    mnl['idx'] = dt['idx']
    mnl['comment'] = message
    mnl['tokenized_comment'] = data['tokenized_comment']
    mnl['tufano_code'] = dt['source']
    mnl['our_code'] = detokenized_code#.replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", "")
    mnl['tokenized_code'] = data['tokenized_code_snippet']
    mnl['written_on'] = dt['written_on']
    
    start_f = detokenized_code.find("\n<|startfocus|>\n")
    end_f = detokenized_code.find("\n<|endfocus|>\n")
    repl_st = detokenized_code[start_f:end_f+15]
    #print(repl_st)
    mnl['our_target'] = target
    '''
    manual_less_then_50.append(dt)
    
from random import shuffle
shuffle(manual_less_then_50)

with open('manual_less_then_50.json', 'w', encoding="utf8") as f:  # writing JSON object
    json.dump(manual_less_then_50, f)
print(len(manual_less_then_50))
#manual_less_then_100[0]


1281
1250


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
import traceback
existing_ids = set()
duplicate_ids = set()
unique_dat = []
i = 0
target_size = 400
error_count = 0
error_list = []

for dt in tufano_less_then_100:
    #print(dt['idx'])
    x = unique_data[dt['idx']]
    i+= 1
    try:
        focuslen = x['tokenized_code_snippet'].index('<|endfocus|>') - x['tokenized_code_snippet'].index('<|startfocus|>')
        key = merge_code_comment(x)
        if key not in existing_ids and len(x['tokenized_target']) <= target_size and focuslen <= target_size:
            existing_ids.add(key)
            unique_dat.append(i-1)
        elif key not in duplicate_ids:
            duplicate_ids.add(key)
    except Exception as e:
        print(e)
        x['error'] = e
        error_list.append(x)
        error_count += 1
        #print(x)
        #break
print(len(unique_dat))

manual_less_then_100 = []
for i in unique_dat: #1:75
    dt = tufano_less_then_100[i]
    '''
    #print(dt['idx'])
    #print(dt['source'])
    data = unique_data[dt['idx']]
    code_file_name = data['code_file_name']
    base_patch_number = data['base_patch_number']
    changed_patch_number = data['changed_patch_number']
    main_code_path = BASE+code_file_name+'/'+str(base_patch_number)+'.java'
    changed_code_path = BASE+code_file_name+'/'+str(changed_patch_number)+'.java'
    line_no = data['base_code_line_number']
    main_code = read(main_code_path)
    changed_code = read(changed_code_path)
    sample = main_code
    message = data['message']
    comment_id = data['comment_id']
    detokenized_code = extreme_detokenization(data['tokenized_code_snippet'])
    #print(detokenized_code.replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", ""))
    mnl['idx'] = dt['idx']
    mnl['comment'] = message
    mnl['tokenized_comment'] = data['tokenized_comment']
    mnl['tufano_code'] = dt['source']
    mnl['our_code'] = detokenized_code#.replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", "")
    mnl['tokenized_code'] = data['tokenized_code_snippet']
    mnl['written_on'] = dt['written_on']
    
    start_f = detokenized_code.find("\n<|startfocus|>\n")
    end_f = detokenized_code.find("\n<|endfocus|>\n")
    repl_st = detokenized_code[start_f:end_f+15]
    #print(repl_st)
    mnl['our_target'] = target
    '''
    manual_less_then_100.append(dt)
    
from random import shuffle
shuffle(manual_less_then_100)
with open('manual_less_then_100.json', 'w', encoding="utf8") as f:  # writing JSON object
    json.dump(manual_less_then_100, f)
print(len(manual_less_then_100))
#manual_less_then_100[0]


1585
1585


In [ ]:
#print(unique_data[0].keys())
print(tufano_less_then_50[0].keys())

In [ ]:
import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
def read(path):
    return open(path, 'r', encoding = 'utf-8').read()


BASE="C:/research_stuff/codes/"
unique_data = getJsonData('E:\\unified_with_date.json')
print(len(unique_data))

In [ ]:
pprint(unique_data[768])

In [27]:
dt = getJsonData('manual_less_then_50.json')
print(len(dt))
print(dt[0]['idx'])
for key in dt[0]:
    if(dt[0][key]!=tufano_less_then_50[0][key]):
        print(dt[0][key])
        print(tufano_less_then_50[0][key])
        print(dt[0]['our_code'])

#print(dt[0]['idx'])
#tufano_less_then_50[0]['idx']

912
37
private static void VAR_1 ( ) { < | VAR_2 | > for ( int i = 0 ; i < < | VAR_3 | > VAR_4 ; i ++ ) { < | VAR_5 | > new VAR_6 ( false ) ; } }
private static void VAR_1 ( ) { for ( int i = 0 ; i < < | VAR_2 | > VAR_3 ; i ++ ) { new VAR_4 ( false ) ; } }
	private static void allocateUnreachableObjects() {
<|startfocus|>
        for(int i = 0; i <<|s|>unreachableObjNum; i++) {
<|endfocus|>
            new MockClass(false);
        }
	}


In [25]:
tufano_less_then_50[0].keys()

dict_keys(['idx', 'project_name', 'written_on', 'tufano_target', 'comment', 'tokenized_comment', 'tufano_code', 'our_target', 'our_code', 'tokenized_code'])

In [58]:
data = unique_data[37]
print(extreme_detokenization(extreme_tokenization(data['source'][2020:2200])))

print(extreme_detokenization(data['tokenized_code_snippet']))

private static void allocateUnreachableObjects() {
<|startfocus|>
        for(int i = 0; i < unreachableObjNum; i++) {
<|endfocus|>
            new MockClass(false);
        }
	}


	private static void allocateUnreachableObjects() {
<|startfocus|>
        for(int i = 0; i <<|s|>unreachableObjNum; i++) {
<|endfocus|>
            new MockClass(false);
        }
	}


# for TUFANO Running

In [115]:
import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
def read(path):
    return open(path, 'r', encoding = 'utf-8').read()


BASE="C:/research_stuff/codes"
unique_data = getJsonData('E:/APR/DATA/unique_data_processed_with_Date_multithread_idx_400_feb_03.json')
#print(unique_data[0])


tufano_test_50 = [13257, 10443, 21152, 8344, 6123, 6122, 21086, 9006, 14137, 2523, 20560, 19915, 11901, 21693, 11107, 2862, 17879, 12346, 14420, 3304, 20658, 13924, 21678, 1945, 19047, 4809, 16525, 21206, 14443, 21481, 5892, 13829, 13432, 5099, 4386, 11997, 19914, 12053, 2593, 10330, 15305, 11388, 3721, 20969, 5081, 1698, 10571, 18025, 15679, 17847, 11928, 11603, 4889, 2275, 2486, 19685, 5751, 8624, 7797, 17666, 7386, 3637, 5322, 15897, 19141, 18152, 9304, 10875, 11265, 10798, 4037, 20348, 1523, 11467, 8681, 3425, 9065, 3218, 10253, 13353, 2653, 14120, 16091, 2254, 7186, 6130, 8591, 9508, 5079, 2852, 13805, 11996, 20144, 13898, 8802, 14704, 18353, 5013, 10893, 21737, 2146, 14135, 8558, 12660, 11119, 14087, 3666, 10010, 21986, 11362, 13777, 14591, 9078, 16886, 11084, 8497, 21973, 3012, 6120, 10343, 11429, 12975, 16540, 6691, 15168, 17916, 12147, 4926, 11437, 13825, 19745, 21110, 2824, 11167, 3949, 10816, 10579, 15102, 13452, 9525, 3361, 5752, 11120, 8512, 17598, 13134, 3389, 17098, 12098, 20887, 12875, 11401, 10811, 19452, 11042, 20528, 18278, 8801, 7395, 7634, 13026, 14750, 12598, 2134, 15328, 19022, 17539, 18903, 20678, 17284, 2698, 19401, 8613, 21682, 13847, 15378, 8760, 19787, 12348, 6266, 5039, 9425, 12811, 16796, 12824, 4066, 8307, 2391, 4808, 17597, 20049, 14315, 13013, 15698, 2851, 2778, 17500, 13803, 6042, 20000, 3333, 18905, 2113, 17724, 8840, 20665, 21543, 10030, 14929, 10877, 14961, 3141, 17846, 13344, 10153, 5940, 12649, 14890, 14759, 21421, 10709, 3897, 3007, 20834, 21107, 7277, 11720, 11016, 3132, 10841, 17746, 20477, 4076, 16242, 21175, 10139, 16452, 10316, 15003, 9490, 20271, 20888, 7093, 19349, 9007, 6101, 11181, 14823, 12253, 1996, 18145, 14751, 9972, 9596, 12088, 19747, 11083, 9567, 12603, 11750, 5546, 4447, 4172, 10231, 1650, 2514, 6627, 18772, 16202, 5078, 13562, 19487, 14082, 11099, 20145, 8017, 2859, 20371, 5300, 4534, 10822, 10089, 7806, 17574, 17865, 7658, 20993, 12764, 18782, 19060, 7917, 17068, 19092, 16548, 14880, 19254, 2046, 5105, 20178, 6343, 12439, 18995, 21093, 7637, 18155, 11978, 17798, 12613, 20820, 21919, 8003, 21297, 6125, 14429, 20179, 15169, 6131, 12874, 18760, 1402, 5474, 20450, 2408, 10968, 4168, 13885, 11620, 10380, 1518, 14043, 2093, 12839, 17671, 1764, 4088, 18904, 2433, 17884, 9189, 8978, 8953, 18781, 1467, 14927, 13354, 16361, 11101, 10873, 18126, 15104, 16655, 9019, 20718, 12065, 14791, 20213, 15955, 17754, 18763, 21618, 21132, 7185, 5684, 4455, 14855, 5415, 7214, 14058, 4021, 18969, 4820, 15936, 8065, 3724, 11624, 16604, 20508, 7767, 15567, 14923, 16698, 6243, 9487, 12865, 2864, 11176, 13093, 2860, 5379, 2879, 12283, 19458, 17392, 7718, 4585, 21436, 17247, 5705, 4735, 2386]
tufano_test_100 = [5465, 15764, 18264, 11017, 17135, 7848, 21592, 15676, 4869, 20105, 15796, 12010, 6956, 13149, 3673, 5991, 21519, 13763, 7374, 16092, 5043, 12551, 14357, 17860, 17187, 13954, 7030, 6202, 3467, 13696, 10996, 10205, 16827, 17144, 7037, 13859, 8467, 14869, 10480, 4893, 8712, 19648, 11063, 10037, 10857, 3957, 14268, 11575, 8316, 3794, 10844, 20940, 11465, 15763, 13030, 6917, 12043, 21798, 17757, 18194, 19258, 4191, 16231, 18081, 9406, 18148, 18118, 15061, 2886, 21260, 11596, 15238, 6756, 12052, 18150, 4086, 7969, 3802, 15128, 4891, 6204, 6094, 21280, 13208, 3800, 12828, 15638, 6190, 5044, 7581, 11108, 7463, 21699, 12292, 12012, 20156, 21525, 16585, 13428, 11331, 11973, 18106, 21605, 4316, 4193, 9350, 8132, 16085, 4081, 17251, 10370, 5124, 12350, 16364, 9090, 8274, 12494, 5494, 4687, 19477, 8572, 16612, 9294, 8711, 4196, 18593, 2124, 7180, 1831, 4438, 19840, 18159, 15787, 16490, 10023, 20247, 18416, 5083, 4697, 18808, 14070, 13494, 8973, 10344, 13591, 19593, 19282, 21673, 12521, 3485, 15253, 5872, 19947, 3465, 10281, 1813, 7368, 13593, 8174, 7208, 6921, 15096, 21011, 9628, 13234, 14074, 14183, 15081, 18103, 8879, 10803, 15959, 13000, 8700, 19198, 5565, 7218, 16129, 11793, 20778, 21560, 5336, 7042, 11242, 18499, 12548, 13612, 5020, 10331, 7885, 2544, 8972, 14452, 3015, 3114, 1748, 9349, 14412, 2565, 4170, 3026, 20135, 21397, 6314, 17076, 20741, 17688, 6694, 16411, 17143, 20033, 10314, 16232, 20161, 3335, 2277, 9411, 2123, 16420, 18312, 4154, 20672, 5019, 8628, 17760, 12749, 20394, 19806, 3058, 12474, 18299, 1820, 15313, 4827, 3370, 15144, 18121, 8573, 4085, 11187, 19948, 17911, 2590, 14870, 11705, 6870, 11654, 16197, 10479, 14051, 12321, 1552, 7239, 2104, 18160, 14986, 10442, 16412, 9450, 10686, 9053, 4821, 6395, 1545, 5694, 8936, 12944, 3977, 7842, 1757, 2644, 6869, 6765, 14451, 9992, 17012, 18331, 6779, 13637, 12173, 10576, 14858, 10516, 7426, 5115, 15713, 2681, 6075, 15603, 21981, 7259, 8503, 7850, 5965, 15727, 8719, 10587, 3630, 20031, 13826, 18817, 9471, 11749, 5048, 11974, 3985, 6758, 9347, 3881, 13521, 5181, 20011, 13934, 13809, 15623, 6211, 17495, 6778, 6191, 15502, 16858, 11014, 7260, 14354, 11856, 2586, 19851, 5356, 20594, 16856, 13276, 7308, 16449, 13305, 15417, 12493, 2888, 9054, 4437, 4892, 19975, 12880, 3911, 14444, 9263, 12051, 7571, 11824, 12135, 21252, 16170, 21654, 15948, 13267, 5495, 6926, 3412, 19478, 5164, 12659, 16047, 2106, 9728, 21580, 12217, 19857, 7608, 15222, 15249, 7613, 19476, 11062, 12684, 2887, 3431, 13733, 12947, 4023, 6955, 11226, 11259, 5726, 11879, 16857, 8814, 16203, 17026, 15139, 12464, 13476, 1926, 14057, 14316, 12627, 4080, 5126, 10382, 1873, 12296, 20347]
global2loval = {}
local2global = {}
for i, d in enumerate(unique_data):
    global2loval[d['global_index']] = i
    local2global[i] = d['global_index']

#"buggy.txt" "fixed.txt"

In [116]:
tufano_less_then_50 = getJsonData("E:/APR/DATA/less_then_50_on_400_v2.json")
tufano_less_then_100 = getJsonData("E:/APR/DATA/less_then_100_on_400_v2.json")

In [117]:
tufano_less_then_50[0].keys()

dict_keys(['idx', 'status', 'global_index', 'project_name', 'written_on', 'tufano_target', 'comment', 'tokenized_comment', 'tufano_code', 'our_target', 'our_code', 'tokenized_code', 'base_code_line_number', 'base_patch_number', 'changed_patch_number', 'code_file_name', 'comment_id', 'class_list', 'func_list', 'prime_var_dic'])

In [118]:
tufano_test_50 = [13257, 10443, 21152, 8344, 6123, 6122, 21086, 9006, 14137, 2523, 20560, 19915, 11901, 21693, 11107, 2862, 17879, 12346, 14420, 3304, 20658, 13924, 21678, 1945, 19047, 4809, 16525, 21206, 14443, 21481, 5892, 13829, 13432, 5099, 4386, 11997, 19914, 12053, 2593, 10330, 15305, 11388, 3721, 20969, 5081, 1698, 10571, 18025, 15679, 17847, 11928, 11603, 4889, 2275, 2486, 19685, 5751, 8624, 7797, 17666, 7386, 3637, 5322, 15897, 19141, 18152, 9304, 10875, 11265, 10798, 4037, 20348, 1523, 11467, 8681, 3425, 9065, 3218, 10253, 13353, 2653, 14120, 16091, 2254, 7186, 6130, 8591, 9508, 5079, 2852, 13805, 11996, 20144, 13898, 8802, 14704, 18353, 5013, 10893, 21737, 2146, 14135, 8558, 12660, 11119, 14087, 3666, 10010, 21986, 11362, 13777, 14591, 9078, 16886, 11084, 8497, 21973, 3012, 6120, 10343, 11429, 12975, 16540, 6691, 15168, 17916, 12147, 4926, 11437, 13825, 19745, 21110, 2824, 11167, 3949, 10816, 10579, 15102, 13452, 9525, 3361, 5752, 11120, 8512, 17598, 13134, 3389, 17098, 12098, 20887, 12875, 11401, 10811, 19452, 11042, 20528, 18278, 8801, 7395, 7634, 13026, 14750, 12598, 2134, 15328, 19022, 17539, 18903, 20678, 17284, 2698, 19401, 8613, 21682, 13847, 15378, 8760, 19787, 12348, 6266, 5039, 9425, 12811, 16796, 12824, 4066, 8307, 2391, 4808, 17597, 20049, 14315, 13013, 15698, 2851, 2778, 17500, 13803, 6042, 20000, 3333, 18905, 2113, 17724, 8840, 20665, 21543, 10030, 14929, 10877, 14961, 3141, 17846, 13344, 10153, 5940, 12649, 14890, 14759, 21421, 10709, 3897, 3007, 20834, 21107, 7277, 11720, 11016, 3132, 10841, 17746, 20477, 4076, 16242, 21175, 10139, 16452, 10316, 15003, 9490, 20271, 20888, 7093, 19349, 9007, 6101, 11181, 14823, 12253, 1996, 18145, 14751, 9972, 9596, 12088, 19747, 11083, 9567, 12603, 11750, 5546, 4447, 4172, 10231, 1650, 2514, 6627, 18772, 16202, 5078, 13562, 19487, 14082, 11099, 20145, 8017, 2859, 20371, 5300, 4534, 10822, 10089, 7806, 17574, 17865, 7658, 20993, 12764, 18782, 19060, 7917, 17068, 19092, 16548, 14880, 19254, 2046, 5105, 20178, 6343, 12439, 18995, 21093, 7637, 18155, 11978, 17798, 12613, 20820, 21919, 8003, 21297, 6125, 14429, 20179, 15169, 6131, 12874, 18760, 1402, 5474, 20450, 2408, 10968, 4168, 13885, 11620, 10380, 1518, 14043, 2093, 12839, 17671, 1764, 4088, 18904, 2433, 17884, 9189, 8978, 8953, 18781, 1467, 14927, 13354, 16361, 11101, 10873, 18126, 15104, 16655, 9019, 20718, 12065, 14791, 20213, 15955, 17754, 18763, 21618, 21132, 7185, 5684, 4455, 14855, 5415, 7214, 14058, 4021, 18969, 4820, 15936, 8065, 3724, 11624, 16604, 20508, 7767, 15567, 14923, 16698, 6243, 9487, 12865, 2864, 11176, 13093, 2860, 5379, 2879, 12283, 19458, 17392, 7718, 4585, 21436, 17247, 5705, 4735, 2386]
tufano_test_100 = [5465, 15764, 18264, 11017, 17135, 7848, 21592, 15676, 4869, 20105, 15796, 12010, 6956, 13149, 3673, 5991, 21519, 13763, 7374, 16092, 5043, 12551, 14357, 17860, 17187, 13954, 7030, 6202, 3467, 13696, 10996, 10205, 16827, 17144, 7037, 13859, 8467, 14869, 10480, 4893, 8712, 19648, 11063, 10037, 10857, 3957, 14268, 11575, 8316, 3794, 10844, 20940, 11465, 15763, 13030, 6917, 12043, 21798, 17757, 18194, 19258, 4191, 16231, 18081, 9406, 18148, 18118, 15061, 2886, 21260, 11596, 15238, 6756, 12052, 18150, 4086, 7969, 3802, 15128, 4891, 6204, 6094, 21280, 13208, 3800, 12828, 15638, 6190, 5044, 7581, 11108, 7463, 21699, 12292, 12012, 20156, 21525, 16585, 13428, 11331, 11973, 18106, 21605, 4316, 4193, 9350, 8132, 16085, 4081, 17251, 10370, 5124, 12350, 16364, 9090, 8274, 12494, 5494, 4687, 19477, 8572, 16612, 9294, 8711, 4196, 18593, 2124, 7180, 1831, 4438, 19840, 18159, 15787, 16490, 10023, 20247, 18416, 5083, 4697, 18808, 14070, 13494, 8973, 10344, 13591, 19593, 19282, 21673, 12521, 3485, 15253, 5872, 19947, 3465, 10281, 1813, 7368, 13593, 8174, 7208, 6921, 15096, 21011, 9628, 13234, 14074, 14183, 15081, 18103, 8879, 10803, 15959, 13000, 8700, 19198, 5565, 7218, 16129, 11793, 20778, 21560, 5336, 7042, 11242, 18499, 12548, 13612, 5020, 10331, 7885, 2544, 8972, 14452, 3015, 3114, 1748, 9349, 14412, 2565, 4170, 3026, 20135, 21397, 6314, 17076, 20741, 17688, 6694, 16411, 17143, 20033, 10314, 16232, 20161, 3335, 2277, 9411, 2123, 16420, 18312, 4154, 20672, 5019, 8628, 17760, 12749, 20394, 19806, 3058, 12474, 18299, 1820, 15313, 4827, 3370, 15144, 18121, 8573, 4085, 11187, 19948, 17911, 2590, 14870, 11705, 6870, 11654, 16197, 10479, 14051, 12321, 1552, 7239, 2104, 18160, 14986, 10442, 16412, 9450, 10686, 9053, 4821, 6395, 1545, 5694, 8936, 12944, 3977, 7842, 1757, 2644, 6869, 6765, 14451, 9992, 17012, 18331, 6779, 13637, 12173, 10576, 14858, 10516, 7426, 5115, 15713, 2681, 6075, 15603, 21981, 7259, 8503, 7850, 5965, 15727, 8719, 10587, 3630, 20031, 13826, 18817, 9471, 11749, 5048, 11974, 3985, 6758, 9347, 3881, 13521, 5181, 20011, 13934, 13809, 15623, 6211, 17495, 6778, 6191, 15502, 16858, 11014, 7260, 14354, 11856, 2586, 19851, 5356, 20594, 16856, 13276, 7308, 16449, 13305, 15417, 12493, 2888, 9054, 4437, 4892, 19975, 12880, 3911, 14444, 9263, 12051, 7571, 11824, 12135, 21252, 16170, 21654, 15948, 13267, 5495, 6926, 3412, 19478, 5164, 12659, 16047, 2106, 9728, 21580, 12217, 19857, 7608, 15222, 15249, 7613, 19476, 11062, 12684, 2887, 3431, 13733, 12947, 4023, 6955, 11226, 11259, 5726, 11879, 16857, 8814, 16203, 17026, 15139, 12464, 13476, 1926, 14057, 14316, 12627, 4080, 5126, 10382, 1873, 12296, 20347]
print(len(tufano_test_50))
test_data_50 = []
test_data_100 = []
for i in tufano_test_50:
    st = unique_data[global2loval[i]]['status']
    if not (st =='unchanged' or st =="none"):
        test_data_50.append(unique_data[global2loval[i]])
for i in tufano_test_100:
    st = unique_data[global2loval[i]]['status']
    if not (st =='unchanged' or st =="none"):
        test_data_100.append(unique_data[global2loval[i]])

400


In [119]:
print(len(test_data_50))
print(len(test_data_100))

338
341


In [37]:
idx_100 = [data['global_index'] for data in test_data_100]
print(idx_100)

[15764, 18264, 11017, 17135, 7848, 21592, 15676, 4869, 20105, 15796, 12010, 6956, 13149, 3673, 5991, 21519, 13763, 7374, 5043, 14357, 17187, 13954, 7030, 6202, 13696, 10996, 10205, 7037, 13859, 8467, 14869, 10480, 4893, 8712, 19648, 11063, 10037, 10857, 14268, 11575, 8316, 3794, 10844, 20940, 11465, 15763, 13030, 6917, 12043, 21798, 18194, 19258, 4191, 16231, 18081, 9406, 18148, 18118, 15061, 2886, 21260, 11596, 15238, 6756, 12052, 4086, 3802, 15128, 4891, 6204, 13208, 3800, 12828, 15638, 6190, 5044, 7581, 11108, 7463, 21699, 12292, 12012, 20156, 21525, 16585, 13428, 11331, 11973, 18106, 21605, 4316, 4193, 9350, 8132, 16085, 4081, 17251, 10370, 5124, 12350, 9090, 8274, 12494, 5494, 4687, 19477, 8572, 16612, 9294, 4196, 2124, 7180, 1831, 4438, 19840, 18159, 15787, 10023, 18416, 5083, 4697, 18808, 14070, 13494, 8973, 10344, 13591, 19593, 19282, 21673, 12521, 3485, 15253, 19947, 3465, 10281, 7368, 13593, 7208, 6921, 15096, 21011, 13234, 14074, 14183, 15081, 18103, 8879, 10803, 13000, 8700

In [120]:
global2loval = {}
local2global = {}
for i, d in enumerate(tufano_less_then_50):
    global2loval[d['global_index']] = i
    local2global[i] = d['global_index']

buggy = open("buggy.txt", "w", encoding='utf-8')
fixed = open("fixed.txt", "w", encoding='utf-8')
for data in test_data_50:
    tufano_data = tufano_less_then_50[global2loval[data['global_index']]]
    buggy.write(tufano_data['tufano_code']+'\n')
    fixed.write(tufano_data['tufano_target']+'\n')
    #pprint(tufano_data)
    #break
buggy.close()
fixed.close()

In [123]:
global2loval = {}
local2global = {}
for i, d in enumerate(tufano_less_then_100):
    global2loval[d['global_index']] = i
    local2global[i] = d['global_index']
print(len(test_data_100))
tufano_100_test = []
for data in test_data_100:
    try:
        tufano_data = tufano_less_then_100[global2loval[data['global_index']]]
        tufano_100_test.append(tufano_data)
    except:
        continue
print(len(tufano_100_test))
import json
with open('tufano_100_test_v2.json', 'w', encoding="utf8") as f:  # writing JSON object
    json.dump(tufano_100_test, f)

341
340


In [46]:
len(tufano_100_test)
tufano_100_test[0]

{'idx': 82089,
 'status': 'update',
 'global_index': 15764,
 'project_name': 'android_',
 'written_on': '2015-03-09 13:09:11',
 'tufano_target': 'protected TYPE_1 METHOD_1 ( long VAR_1 ) { TYPE_2 VAR_2 = new TYPE_2 ( VAR_3 , VAR_4 ) ; VAR_2 . METHOD_2 ( VAR_1 ) ; VAR_2 . METHOD_3 ( 0 ) ; VAR_2 . METHOD_3 ( 1 ) ; TYPE_3 VAR_5 = VAR_6 . METHOD_4 ( VAR_2 ) ; VAR_6 . METHOD_5 ( VAR_5 ) ; int VAR_7 = VAR_5 . METHOD_6 ( ) ; assertEquals ( STRING_1 , 1 , VAR_7 ) ; VAR_5 . METHOD_7 ( ) ; return VAR_5 . METHOD_8 ( ) ; }',
 'comment': 'Avoid the loc temporary?',
 'tokenized_comment': ['Avoid',
  '<|s|>',
  'the',
  '<|s|>',
  'loc',
  '<|s|>',
  'temporary',
  '?'],
 'tufano_code': 'protected TYPE_1 METHOD_1 ( long VAR_1 ) { TYPE_2 VAR_2 = new TYPE_2 ( VAR_3 , VAR_4 ) ; VAR_2 . METHOD_2 ( VAR_1 ) ; VAR_2 . METHOD_3 ( 0 ) ; VAR_2 . METHOD_3 ( 1 ) ; TYPE_3 VAR_5 = VAR_6 . METHOD_4 ( VAR_2 ) ; VAR_6 . METHOD_5 ( VAR_5 ) ; int VAR_7 = VAR_5 . METHOD_6 ( ) ; assertEquals ( STRING_1 , 1 , VAR_7 ) ; VA

In [143]:
p=p+1
#p=170
p = 47
print(tufano_50_test[p]['tufano_code'])
print("##########################")
print(tufano_50_test[p]['our_code'])


public void VAR_1 ( VAR_2 listener ) { VAR_3 . add ( listener ) ; } }
##########################
<|startfocus|>
  public void addListener(CaptureListener listener) {
    myListeners.add(listener);
  }
<|endfocus|>
}


In [145]:
tufano_50_test[0]

{'idx': 17004,
 'status': 'update',
 'global_index': 21152,
 'project_name': 'android_',
 'written_on': '2013-10-01 00:29:37',
 'tufano_target': 'byte [ ] METHOD_5 ( TYPE_1 VAR_3 ) { if ( ! ( VAR_3 instanceof TYPE_2 ) ) { return null ; } TYPE_2 VAR_4 = ( TYPE_2 ) VAR_3 ; try { TYPE_3 VAR_5 = new TYPE_3 ( ) ; TYPE_4 VAR_6 = new TYPE_4 ( VAR_5 ) ; VAR_6 . METHOD_6 ( VAR_7 ) ; byte [ ] data = VAR_4 . METHOD_7 ( ) ; VAR_6 . METHOD_6 ( data . length ) ; VAR_6 . METHOD_8 ( data ) ; TYPE_5 [ ] VAR_8 = VAR_3 . METHOD_9 ( ) ; VAR_6 . METHOD_6 ( VAR_8 . length ) ; for ( TYPE_5 VAR_9 : VAR_8 ) { data = VAR_9 . METHOD_7 ( ) ; VAR_6 . METHOD_6 ( data . length ) ; VAR_6 . METHOD_8 ( data ) ; } TYPE_6 < byte [ ] > VAR_10 = VAR_4 . METHOD_10 ( ) ; VAR_6 . METHOD_6 ( VAR_10 . size ( ) ) ; for ( byte [ ] VAR_11 : VAR_10 ) { VAR_6 . METHOD_6 ( VAR_11 . length ) ; VAR_6 . METHOD_8 ( VAR_11 ) ; } byte [ ] VAR_12 = VAR_4 . METHOD_11 ( ) ; if ( VAR_12 != null ) { VAR_6 . METHOD_6 ( VAR_12 . length ) ; VAR_6 

In [152]:
java_cmd = "java -jar src2abs-0.1-jar-with-dependencies.jar pair method "
java_cmd = java_cmd + "temp\\temp.java temp\\temp2.java"+" "
java_cmd = java_cmd+"temp\\processed.java temp\\processed2.java"+' idioms.csv'
error_cnt = 0
final_50_test = []
for i in range(len(tufano_50_test)):
    
    data = tufano_50_test[i]
    detokenized_code = extreme_detokenization(data['tokenized_code'])
    #print(detokenized_code)
    start_f = detokenized_code.find("<|startfocus|>\n")
    end_f = detokenized_code.find("<|endfocus|>\n")
    #print("se ", start_f, " ", end_f)
    repl_st = detokenized_code[start_f:end_f+14]
    
    s1 = extreme_detokenization(data['tokenized_code']).replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", "")
    s2 = extreme_detokenization(data['tokenized_code']).replace(repl_st, data['our_target'].replace("\n<|del|>\n", ""))
    
    with open("temp\\temp.java", 'w', encoding='utf-8') as f:
        s1 = s1.replace("@Nullable ", "").replace("@Override", "").replace("@Nonnull ", "").replace("@NotNull ", "").replace("@NonNull ", "").replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", "").strip()
        f.write(s1)
    with open("temp\\temp2.java", 'w', encoding='utf-8') as f:
        s2 = s2.replace("@Nullable ", "").replace("@Override", "").replace("@Nonnull ", "").replace("@NotNull ", "").replace("@NonNull ", "").replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", "").strip()
        f.write(s2)
    check = subprocess.check_output(java_cmd.replace('/', '\\'),stderr=subprocess.STDOUT, shell=True)
    if "Parsing ERROR" in str(check):
        error_cnt+=1
        print("################    "," index =" , i, "     ##############")
        #print(s)
        print(tufano_50_test[i]['our_code'])
    else:
        tufano_50_test[i]['tufano_code'] = read("temp\\processed.java")
        tufano_50_test[i]['tufano_target'] = read("temp\\processed2.java")
        final_50_test.append(tufano_50_test[i])
print(error_cnt)
print(len(final_50_test))

################      index = 5      ##############
public Boolean eval(U tested, Context context) {
    boolean leftValue = left.eval(tested, context).booleanValue();
    boolean rightValue = right.eval(tested, context).booleanValue();
    boolean result;
    switch (operator) {
      case OR:
        result = leftValue || rightValue;
        break;
      case AND:
        result = leftValue && rightValue;
        break;
      case XOR:
        result = leftValue ^ rightValue;
        break;
      case EQUALS:
        result = leftValue == rightValue;
        break;
      default:
        throw new AssertionError();
    }
    return Boolean.valueOf(result);
  }
    public String getXmlString(ResourceType type, boolean system) {
        if (type == ResourceType.ID /* && isDeclaredInline()*/) {
<|startfocus|>
            return (system ? "@android:" : "@+") + type.getName() + "/" + getName();
<|endfocus|>
        }

        return (system ? "@android:" : "@") + type.getName() + "/" + ge

  }
    private String displayName(String id) {
<|startfocus|>
        // Android changed: Android doesn't have long names for GMT/UTC yet.
<|endfocus|>
//        if (id.equals("GMT")) {
//            return "Greenwich Mean Time";
//        }
//        if (id.equals("GMT0")) {
//            return "Greenwich Mean Time";
//        }
//        if (id.equals("UTC")) {
//            return "Coordinated Universal Time";
//        }
        return id;
    }
################      index = 89      ##############
public Boolean eval(U tested, Context context) {
    boolean leftValue = left.eval(tested, context).booleanValue();
    boolean rightValue = right.eval(tested, context).booleanValue();
    boolean result;
    switch (operator) {
      case OR:
        result = leftValue || rightValue;
        break;
      case AND:
        result = leftValue && rightValue;
        break;
      case XOR:
        result = leftValue ^ rightValue;
        break;
      case EQUALS:
        result = leftValue

  }
  public UsersListItem (User aUser) {
<|startfocus|>
    super();
<|endfocus|>
    myUser = aUser;
  }
################      index = 128      ##############
public Boolean eval(U tested, Context context) {
    boolean leftValue = left.eval(tested, context).booleanValue();
    boolean rightValue = right.eval(tested, context).booleanValue();
    boolean result;
    switch (operator) {
      case OR:
        result = leftValue || rightValue;
        break;
      case AND:
        result = leftValue && rightValue;
        break;
      case XOR:
        result = leftValue ^ rightValue;
        break;
      case EQUALS:
        result = leftValue == rightValue;
        break;
      default:
        throw new AssertionError();
    }
    return Boolean.valueOf(result);
  }
        EmnReceiver emn = new EmnReceiver();

        assertEquals("email@address.com", emn.decode(fromHexString(EMAIL_FIRST)));
        assertEquals("email@address.com", emn.decode(fromHexString(TIMESTAMP_FIRST)));
    

  }
    protected void tearDown() {
        // Reset the uncaughtExceptionHandler to what it was when the test began.
        Thread.setDefaultUncaughtExceptionHandler(originalThreadDefaultUncaughtExceptionHandler);
<|startfocus|>
        try {
            // Give the threads a chance to die.
            Thread.sleep(50);
        } catch (InterruptedException e) {
        }
<|endfocus|>
    }
################      index = 229      ##############
public Boolean eval(U tested, Context context) {
    boolean leftValue = left.eval(tested, context).booleanValue();
    boolean rightValue = right.eval(tested, context).booleanValue();
    boolean result;
    switch (operator) {
      case OR:
        result = leftValue || rightValue;
        break;
      case AND:
        result = leftValue && rightValue;
        break;
      case XOR:
        result = leftValue ^ rightValue;
        break;
      case EQUALS:
        result = leftValue == rightValue;
        break;
      default:
        throw

        }
################      index = 294      ##############
public Boolean eval(U tested, Context context) {
    boolean leftValue = left.eval(tested, context).booleanValue();
    boolean rightValue = right.eval(tested, context).booleanValue();
    boolean result;
    switch (operator) {
      case OR:
        result = leftValue || rightValue;
        break;
      case AND:
        result = leftValue && rightValue;
        break;
      case XOR:
        result = leftValue ^ rightValue;
        break;
      case EQUALS:
        result = leftValue == rightValue;
        break;
      default:
        throw new AssertionError();
    }
    return Boolean.valueOf(result);
  }
    public void dispose() {
        //Remove all messages from the queue
        this.removeCallbacksAndMessages(null);

<|startfocus|>
        this.phone = null;
<|endfocus|>
    }
################      index = 296      ##############
public Boolean eval(U tested, Context context) {
    boolean leftValue = left.eva

In [154]:
#print(final_50_test[1]['tufano_code'].split(" "))
#final_50_test[1]['tufano_target']
final_50_test[2]

{'idx': 10613,
 'status': 'update',
 'global_index': 21086,
 'project_name': 'android_',
 'written_on': '2013-09-09 23:08:21',
 'tufano_target': 'public static String METHOD_4 ( TYPE_1 file ) { return METHOD_4 ( file , true ) ; }',
 'comment': 'In a finally block, call Closeables.close().',
 'tokenized_comment': ['In',
  '<|s|>',
  'a',
  '<|s|>',
  'finally',
  '<|s|>',
  'block',
  ',',
  '<|s|>',
  'call',
  '<|s|>',
  'Closeables',
  '.',
  'close',
  '(',
  ')',
  '.'],
 'tufano_code': 'public static void METHOD_1 ( TYPE_1 file , String VAR_1 ) { try { TYPE_2 writer = new TYPE_2 ( new TYPE_3 ( file ) ) ; writer.write ( VAR_1 ) ; writer . METHOD_2 ( ) ; } catch ( TYPE_4 VAR_2 ) { TYPE_5 . METHOD_3 ( VAR_2 ) ; } }',
 'our_target': '  @Nullable\n  public static String readTextFile(@NotNull File file) {\n    return readTextFile(file, true);\n',
 'our_code': '<|startfocus|>\n  public static void writeTextFile(@NotNull File file, @NotNull String contents) {\n    try {\n      BufferedWri

In [155]:
with open('final_50_test.json', 'w', encoding="utf8") as f:  # writing JSON object
    json.dump(final_50_test, f)

In [165]:
print(len(final_50_test))
print(len(final_100_test))
#i=0
i = i+10
print(final_100_test[i]['our_code'])
print("#######")
print(final_100_test[i]['tufano_code'])

300
278
    protected void checkInvokeReply(ReplyPacket reply) {
        // Check result is 'void'
        {
            TaggedObject invokeNewObject = reply.getNextValueAsTaggedObject();
            assertEquals(JDWPConstants.Tag.OBJECT_TAG, invokeNewObject.tag);
            assertFalse("Invalid exception object id", invokeNewObject.objectID == 0);
        }

        // Check exception is null.
<|startfocus|>
        {
            TaggedObject invokeException = reply.getNextValueAsTaggedObject();
            assertEquals(JDWPConstants.Tag.OBJECT_TAG, invokeException.tag);
            assertEquals("Invalid exception object id", 0, invokeException.objectID);
        }
<|endfocus|>

        assertAllDataRead(reply);

    }
#######
protected void METHOD_1 ( TYPE_1 VAR_1 ) { { TYPE_2 VAR_2 = VAR_1 . METHOD_2 ( ) ; assertEquals ( VAR_3 , VAR_4 ) ; METHOD_3 ( STRING_1 , VAR_5 == 0 ) ; } { TYPE_2 VAR_6 = VAR_1 . METHOD_2 ( ) ; assertEquals ( VAR_3 , VAR_7 ) ; assertEquals ( STRING_1 , 0 , VAR

In [159]:
unique_data[4023]

{'status': 'update',
 'message': 'nit: remove the else part of this statement',
 'comment_id': 'c8b21311_9ce40536',
 'target': '            if (numEglConfigs[0] <= 0) {\n',
 'code_snippet': '<|startfocus|>\n            else if (numEglConfigs[0] <= 0) {\n<|endfocus|>\n                Slog.e(TAG, "no valid config found");\n                return false;\n            }',
 'prime_var_dic': {'int': 5, 'texture': 1},
 'class_list': ['ColorFade', 'NaturalSurfaceLayout'],
 'func_list': ['public ColorFade(int displayId)',
  'public boolean prepare(Context context, int mode)',
  'private String readFile(Context context, int resourceId)',
  'private int loadShader(Context context, int resourceId, int type)',
  'private boolean initGLShaders(Context context)',
  'private void destroyGLShaders()',
  'private boolean initGLBuffers()',
  'private void destroyGLBuffers()',
  'private static void setQuad(FloatBuffer vtx, float x, float y, float w, float h)',
  'public void dismissResources()',
  'public

In [160]:
global2loval[6042]

KeyError: 6042

# TUFANO ANALYSIS

In [177]:
correct_list =  [36, 159, 237, 243]
changed_list =  [0, 1, 2, 3, 4, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 40, 41, 42, 44, 45, 46, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 118, 119, 120, 121, 122, 123, 124, 125, 126, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 160, 161, 162, 163, 164, 165, 166, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 199, 200, 201, 202, 203, 204, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 226, 227, 228, 229, 232, 233, 234, 235, 236, 238, 239, 240, 241, 242, 244, 245, 246, 247, 248, 249, 250, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 270, 271, 272, 273, 274, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 299]
bad_list =  [5, 9, 25, 26, 39, 43, 47, 49, 70, 73, 82, 105, 116, 117, 127, 138, 167, 185, 198, 205, 225, 230, 231, 251, 269, 275, 297, 298]
'''
global2loval = {}
local2global = {}
for i, d in enumerate(tufano_less_then_50):
    global2loval[d['global_index']] = i
    local2global[i] = d['global_index']

buggy = open("buggy.txt", "w", encoding='utf-8')
fixed = open("fixed.txt", "w", encoding='utf-8')
for data in test_data_50:
    tufano_data = tufano_less_then_50[global2loval[data['global_index']]]
    buggy.write(tufano_data['tufano_code']+'\n')
    fixed.write(tufano_data['tufano_target']+'\n')
    #pprint(tufano_data)
    #break
buggy.close()
fixed.close()
'''
print(final_50_test[36]["comment"], " ", final_50_test[159]["comment"])
#for key in final_50_test[36].keys():
#    print(key, " ", final_50_test[36][key]==final_50_test[159][key])
print(final_50_test[0].keys())
for idx in correct_list:
    print(final_50_test[idx]['our_code'])
    print("###        ####")
    print(final_50_test[idx]['our_target'])
    print("#####################")

ok, fixed with the next upload

   Intendation is incorrect.
dict_keys(['idx', 'status', 'global_index', 'project_name', 'written_on', 'tufano_target', 'comment', 'tokenized_comment', 'tufano_code', 'our_target', 'our_code', 'tokenized_code', 'base_code_line_number', 'base_patch_number', 'changed_patch_number', 'code_file_name', 'comment_id', 'class_list', 'func_list', 'prime_var_dic'])
    protected void onCreate(Bundle icicle) {
     super.onCreate(icicle);
        
<|startfocus|>
     addPreferencesFromResource(R.xml.gsm_umts_options);
<|endfocus|>
    }
###        ####
        addPreferencesFromResource(R.xml.gsm_umts_options);

#####################
    protected void onCreate(Bundle icicle) {
     super.onCreate(icicle);
        
<|startfocus|>
     addPreferencesFromResource(R.xml.gsm_umts_options);
<|endfocus|>
    }
###        ####
        addPreferencesFromResource(R.xml.gsm_umts_options);

#####################
    public static TreeViewModel getModel() {
        if (model =

In [166]:

buggy = open("buggy.txt", "w", encoding='utf-8')
fixed = open("fixed.txt", "w", encoding='utf-8')
for data in final_50_test:
    #tufano_data = tufano_less_then_50[global2loval[data['global_index']]]
    buggy.write(data['tufano_code']+'\n')
    fixed.write(data['tufano_target']+'\n')
    
buggy.close()
fixed.close()

In [181]:
print(len(final_50_test))
same = 0
for data in final_50_test:
    if(data['tufano_target']==data['tufano_code']):
        same+=1
        continue
    print(data['tufano_code'])
    print("#############     ###############")
    print(data['tufano_target'])
    print("#################################")
print(same)

300
private void METHOD_1 ( ) { if ( VAR_1 . METHOD_2 ( ) ) { return ; } VAR_1 . METHOD_3 ( ) ; VAR_1 . METHOD_4 ( VAR_2 , INT_1 ) ; }
#############     ###############
private void METHOD_1 ( ) { if ( VAR_1 . METHOD_2 ( ) ) { return ; } VAR_1 . METHOD_3 ( ) ; VAR_1 . METHOD_4 ( VAR_2 , VAR_3 ) ; }
#################################
public static void METHOD_1 ( TYPE_1 file , String VAR_1 ) { try { TYPE_2 writer = new TYPE_2 ( new TYPE_3 ( file ) ) ; writer.write ( VAR_1 ) ; writer . METHOD_2 ( ) ; } catch ( TYPE_4 VAR_2 ) { TYPE_5 . METHOD_3 ( VAR_2 ) ; } }
#############     ###############
public static String METHOD_4 ( TYPE_1 file ) { return METHOD_4 ( file , true ) ; }
#################################
public void METHOD_1 ( ) { super . METHOD_1 ( ) ; if ( ! VAR_1 && TYPE_1 . METHOD_2 ( VAR_2 ) == 0 && VAR_3 ) { TYPE_2 . METHOD_3 ( this , STRING_1 ) ; return ; } METHOD_4 ( VAR_3 ) ; }
#############     ###############
public void METHOD_1 ( ) { super . METHOD_1 ( ) ; if ( ! VAR_1 &

public boolean METHOD_1 ( String VAR_1 , String fileName ) { if ( VAR_1 == null || fileName == null ) { TYPE_1 . METHOD_3 ( VAR_3 , STRING_1 ) ; return false ; } return VAR_2 . METHOD_2 ( VAR_1 , fileName ) ; }
#################################
public TYPE_1 METHOD_1 ( String VAR_1 , int VAR_2 ) { if ( VAR_1 == null ) { throw new TYPE_2 ( STRING_1 ) ; } synchronized ( VAR_3 ) { return VAR_3 . METHOD_2 ( VAR_1 , VAR_2 ) ; } }
#############     ###############
public TYPE_1 METHOD_1 ( String VAR_1 ) { if ( VAR_1 == null ) { throw new TYPE_2 ( STRING_1 ) ; } synchronized ( VAR_3 ) { return VAR_3 . METHOD_2 ( VAR_1 , VAR_2 ) ; } }
#################################
public TYPE_1 < TYPE_2 > METHOD_1 ( ) { synchronized ( VAR_1 ) { TYPE_1 < TYPE_2 > VAR_2 = new TYPE_3 ( ) ; VAR_2 . METHOD_2 ( VAR_1 ) ; return TYPE_4 . METHOD_3 ( VAR_2 ) ; } }
#############     ###############
public TYPE_1 < TYPE_2 > METHOD_1 ( ) { synchronized ( VAR_1 ) { return TYPE_4 . METHOD_3 ( ( TYPE_1 < TYPE_2 > ) VAR_1

private boolean METHOD_1 ( ) { long VAR_1 = VAR_2 . get ( ) ; METHOD_2 ( VAR_3 ) ; return ! ( METHOD_3 ( ) && VAR_2 . get ( ) == VAR_1 && VAR_4 != null ) ; }
#############     ###############
private TYPE_1 < ? > METHOD_1 ( ) { long VAR_1 = VAR_2 . get ( ) ; METHOD_2 ( VAR_3 ) ; return ! ( METHOD_3 ( ) && VAR_2 . get ( ) == VAR_1 && VAR_4 != null ) ; }
#################################
public boolean equals ( TYPE_1 obj ) { return obj instanceof TYPE_2 && ( ( TYPE_2 ) obj ) . VAR_1 . equals ( VAR_1 ) && ( ( TYPE_2 ) obj ) . VAR_2 . equals ( VAR_2 ) ; }
#############     ###############
public final boolean equals ( TYPE_1 obj ) { return obj instanceof TYPE_2 && ( ( TYPE_2 ) obj ) . VAR_1 . equals ( VAR_1 ) && ( ( TYPE_2 ) obj ) . VAR_2 . equals ( VAR_2 ) ; }
#################################
public VAR_1 ( String VAR_2 , String id ) { this . METHOD_1 ( VAR_2 ) ; this . METHOD_2 ( id ) ; }
#############     ###############
public VAR_1 ( String VAR_2 , String id ) { METHOD_1 ( VAR_2 ) ;

#################################
public void METHOD_1 ( TYPE_1 < String , TYPE_2 > VAR_1 ) { synchronized ( VAR_2 ) { VAR_2 . METHOD_2 ( ) ; VAR_2 . METHOD_3 ( VAR_1 ) ; } }
#############     ###############
public void METHOD_1 ( TYPE_1 < String , TYPE_2 > VAR_1 ) { synchronized ( this ) { VAR_2 . METHOD_2 ( ) ; VAR_2 . METHOD_3 ( VAR_1 ) ; } }
#################################
public void METHOD_1 ( ) { try { TYPE_1 . METHOD_2 ( STRING_1 ) ; fail ( STRING_2 ) ; } catch ( TYPE_2 VAR_1 ) { } }
#############     ###############
public void METHOD_1 ( ) { try { TYPE_1 . METHOD_2 ( STRING_3 ) ; fail ( STRING_2 ) ; } catch ( TYPE_2 VAR_1 ) { } }
#################################
private boolean VAR_1 = false ; public boolean METHOD_1 ( ) { return VAR_1 ; }
#############     ###############
private boolean VAR_2 = false ; public boolean METHOD_1 ( ) { return VAR_1 ; }
#################################
private static boolean METHOD_1 ( ) { return TYPE_1 . METHOD_2 ( ) . METHOD_3 ( ) . getBo

static void METHOD_1 ( TYPE_1 VAR_1 ) { TYPE_2 . METHOD_2 ( VAR_7 ) ; TYPE_2 . METHOD_2 ( VAR_3 , VAR_4 ) ; TYPE_2 . METHOD_2 ( VAR_5 , VAR_6 ) ; }
#################################
private boolean METHOD_1 ( ) { TYPE_1 VAR_1 = ( TYPE_1 ) VAR_2 . getContext ( ) . METHOD_2 ( VAR_3 ) ; return VAR_1 . METHOD_3 ( ) ; }
#############     ###############
private boolean METHOD_1 ( ) { final TYPE_1 VAR_1 = ( TYPE_1 ) VAR_2 . getContext ( ) . METHOD_2 ( VAR_3 ) ; return VAR_1 . METHOD_3 ( ) ; }
#################################
public String getName ( ) { if ( METHOD_1 ( ) == null ) { return null ; } return TYPE_1 . METHOD_2 ( METHOD_1 ( ) ) ; }
#############     ###############
public String getName ( ) { TYPE_2 VAR_1 = METHOD_1 ( ) ; return VAR_1 == null ? null : TYPE_1 . METHOD_2 ( VAR_1 ) ; }
#################################
public TYPE_1 METHOD_1 ( ) { return null ; }
#############     ###############
public TYPE_1 METHOD_1 ( ) { return VAR_1 ; }
#################################
public 

In [185]:
file_buggy = open('E:/APR/RepairFromReviewTrainingData/tufano_train_data/buggy.txt', 'r')
file_fixed = open('E:/APR/RepairFromReviewTrainingData/tufano_train_data/fixed.txt', 'r')
same = 0
total = 0
for line in file_buggy:
    total+=1
    buggy_line = line.strip()
    fixed_line = file_fixed.readline().strip()
    if(buggy_line==fixed_line):
        same+=1
        continue
    print(buggy_line)
    print("############     ###############")
    print(fixed_line)
    print("################################")
print(same)   
print(total)

private void METHOD_1 ( ) { VAR_1 . METHOD_2 ( STRING_1 , VAR_2 , VAR_3 ) ; TYPE_1 . METHOD_1 ( ( VAR_2 ? VAR_3 : 0 ) ) ; }
############     ###############
private void METHOD_1 ( ) { TYPE_1 . METHOD_1 ( ( VAR_2 ? VAR_3 : 0 ) ) ; }
################################
public TYPE_1 METHOD_1 ( java.lang.String input ) throws TYPE_2 { try { return TYPE_1 . METHOD_2 ( input ) ; } catch ( java.lang.IllegalArgumentException e ) { throw new TYPE_2 ( java.lang.String.format ( STRING_1 , e.getMessage ( ) ) , e ) ; } }
############     ###############
public java.lang.String METHOD_1 ( java.lang.String input ) { return input ; }
################################
public void METHOD_1 ( ) { android.os.Message m = VAR_1 . METHOD_2 ( VAR_2 ) ; VAR_1 . METHOD_3 ( m ) ; }
############     ###############
public void METHOD_1 ( ) { VAR_1 . METHOD_3 ( VAR_2 ) ; }
################################
public void METHOD_1 ( ) throws java.lang.Exception { TYPE_1 . METHOD_2 ( VAR_1 , null , null , 0 , false ) ; }


public static void METHOD_1 ( TYPE_1 args , TYPE_2 t ) { t . METHOD_2 ( VAR_1 , VAR_1 , true ) ; args.out = TYPE_3 . METHOD_3 ( args . inV , t ) ; }
################################
public void METHOD_1 ( int VAR_1 , java.lang.String VAR_2 ) { }
############     ###############
public void error ( int VAR_1 , java.lang.String VAR_2 ) throws android.os.RemoteException { }
################################
public TYPE_1 METHOD_1 ( ) { return VAR_1 . METHOD_2 ( ) ? null : new TYPE_1 ( new java.util.ArrayList < java.lang.String > ( VAR_1 . METHOD_3 ( ) ) ) ; }
############     ###############
public java.util.Set < java.lang.String > METHOD_3 ( ) { return VAR_1 . METHOD_3 ( ) ; }
################################
public void METHOD_1 ( TYPE_1 VAR_1 ) { android.util.Log.e ( VAR_2 . TAG , ( STRING_1 + ( VAR_1 . toString ( ) ) ) ) ; }
############     ###############
public void METHOD_1 ( TYPE_1 VAR_1 ) { }
################################
public java.lang.String METHOD_1 ( ) { return ( ( ( na

public static TYPE_1 METHOD_1 ( ) { TYPE_2 VAR_1 = TYPE_2 . METHOD_2 ( ) ; return VAR_1 instanceof TYPE_1 ? ( ( TYPE_1 ) ( VAR_1 ) ) : null ; }
############     ###############
public static TYPE_1 METHOD_1 ( ) { TYPE_2 VAR_1 = TYPE_2 . METHOD_2 ( ) ; if ( VAR_1 instanceof TYPE_1 ) { return new TYPE_1 ( ( ( TYPE_1 ) ( VAR_1 ) ) ) ; } return null ; }
################################
public void METHOD_1 ( java.lang.Integer VAR_1 , java.lang.String VAR_2 ) { VAR_3 . METHOD_2 ( VAR_1 , TYPE_1 . METHOD_3 ( VAR_2 ) ) ; }
############     ###############
public void METHOD_1 ( java.lang.Integer VAR_1 , byte [ ] VAR_2 ) { VAR_3 . METHOD_2 ( VAR_1 , VAR_2 ) ; }
################################
public void METHOD_1 ( ) { assertTrue ( VAR_1 . METHOD_2 ( STRING_1 , STRING_2 ) ) ; assertTrue ( VAR_1 . METHOD_2 ( STRING_3 , STRING_4 ) ) ; assertFalse ( VAR_1 . METHOD_2 ( STRING_5 , STRING_4 ) ) ; }
############     ###############
public void METHOD_1 ( ) { assertTrue ( VAR_1 . METHOD_2 ( STRING_1 

public void METHOD_1 ( TYPE_1 VAR_1 ) { if ( VAR_2 ) { TYPE_2 . log ( STRING_1 ) ; } METHOD_2 ( ) ; }
############     ###############
public void METHOD_1 ( TYPE_1 VAR_1 ) { METHOD_2 ( ) ; }
################################
public void METHOD_1 ( ) throws java.lang.Exception { TYPE_1 VAR_1 = new TYPE_1 ( ) ; VAR_1 . METHOD_2 ( 1 , 1 , 1 ) ; }
############     ###############
public void METHOD_1 ( ) throws java.lang.Exception { try ( TYPE_1 VAR_1 = new TYPE_1 ( ) ) { VAR_1 . METHOD_2 ( 1 , 1 , 1 ) ; } }
################################
public void METHOD_1 ( java.io.File VAR_1 ) throws TYPE_1 { VAR_2 = VAR_1 ; super . METHOD_1 ( VAR_1 ) ; }
############     ###############
public void METHOD_1 ( java.lang.Object VAR_1 ) throws TYPE_1 { VAR_2 = ( ( java.io.File ) ( VAR_1 ) ) ; super . METHOD_1 ( VAR_1 ) ; }
################################
static void METHOD_1 ( TYPE_1 < ? > VAR_1 ) { VAR_1 . METHOD_2 ( ) ; if ( VAR_1 != null ) { if ( VAR_2 ) { throw new TYPE_2 ( STRING_1 ) ; } } }
###

static void METHOD_1 ( TYPE_1 < ? > VAR_1 ) { if ( VAR_1 != null ) { Main . METHOD_2 ( null ) ; Main . METHOD_2 ( VAR_1 ) ; } }
################################
private void METHOD_1 ( TYPE_1 in ) throws java.io.IOException { java.lang.String line = METHOD_2 ( in ) ; if ( ! ( line . METHOD_3 ( ) ) ) { throw new TYPE_2 ( ( STRING_1 + line ) ) ; } }
############     ###############
private void METHOD_1 ( TYPE_1 in ) throws java.io.IOException { java.lang.String line = METHOD_2 ( in ) ; if ( ( line . length ( ) ) != 0 ) { throw new TYPE_2 ( ( STRING_1 + line ) ) ; } }
################################
public void METHOD_1 ( TYPE_1 VAR_1 , TYPE_2 VAR_2 , TYPE_2 VAR_3 , TYPE_3 VAR_4 , TYPE_2 VAR_5 , TYPE_4 callback , TYPE_5 VAR_6 ) { METHOD_1 ( VAR_1 , VAR_2 , VAR_3 , VAR_4 , VAR_5 , 0 , 0 , callback , VAR_6 ) ; }
############     ###############
public void METHOD_1 ( TYPE_1 VAR_1 , TYPE_2 VAR_2 , TYPE_2 VAR_3 , TYPE_3 VAR_4 , TYPE_2 VAR_5 , TYPE_4 callback , TYPE_5 VAR_6 ) { METHOD_1 ( VA

...

public boolean METHOD_1 ( java.lang.String VAR_1 , java.lang.String VAR_2 , org.ovirt.engine.core.compat . Version VAR_3 ) { final TYPE_1 VAR_4 = VAR_5 . get ( VAR_3 ) ; if ( VAR_4 != null ) { return VAR_4 . METHOD_2 ( VAR_1 , VAR_2 ) ; } return false ; }
############     ###############
public boolean METHOD_1 ( java.lang.String VAR_1 , java.lang.String VAR_2 , org.ovirt.engine.core.compat . Version VAR_3 ) { final TYPE_1 VAR_4 = VAR_5 . get ( VAR_3 ) ; if ( VAR_4 != null ) { return VAR_4 . METHOD_1 ( VAR_1 , VAR_2 ) ; } return false ; }
################################
public static TYPE_1 < TYPE_2 . ValidationResult > METHOD_1 ( TYPE_3 . VdcBllMessages VAR_1 , TYPE_4 < java.lang.String > VAR_2 ) { return new TYPE_5 ( VAR_1 , VAR_2 . METHOD_2 ( new java.lang.String [ VAR_2 . size ( ) ] ) ) ; }
############     ###############
public static TYPE_1 < TYPE_2 . ValidationResult > METHOD_1 ( TYPE_3 . EngineMessage VAR_1 , TYPE_4 < java.lang.String > VAR_2 ) { return new TYPE_5 ( VAR_1 , V

private boolean METHOD_1 ( ) { return ! ( getReturnValue ( ) . METHOD_2 ( ) . isEmpty ( ) ) ; }
################################
protected void METHOD_1 ( ) { TYPE_1 . METHOD_2 ( ) . METHOD_3 ( TYPE_2 . getInstance ( ) . METHOD_4 ( ) . METHOD_5 ( TYPE_1 . getParameters ( ) . METHOD_6 ( ) ) ) ; }
############     ###############
protected void METHOD_1 ( ) { TYPE_1 . METHOD_2 ( ) . METHOD_3 ( org.ovirt.engine.core.dal.dbbroker.DbFacade.getInstance ( ) . METHOD_4 ( ) . METHOD_5 ( TYPE_1 . getParameters ( ) . METHOD_6 ( ) ) ) ; }
################################
public void METHOD_1 ( ) { TYPE_1 VAR_1 = VAR_2 . METHOD_2 ( ) . get ( new org.ovirt.engine.core.compat.Guid ( STRING_1 ) ) ; TYPE_2 . METHOD_3 ( VAR_1 ) ; }
############     ###############
public void METHOD_1 ( ) { TYPE_1 VAR_1 = dao . get ( new org.ovirt.engine.core.compat.Guid ( STRING_1 ) ) ; TYPE_2 . METHOD_3 ( VAR_1 ) ; }
################################
public static TYPE_1 METHOD_1 ( TYPE_2 VAR_1 ) { return VAR_1 . METHO

public void METHOD_1 ( ) { METHOD_2 ( ) ; TYPE_1 . METHOD_3 ( METHOD_4 ( ) . METHOD_5 ( ) ) ; }
############     ###############
public void METHOD_1 ( ) { TYPE_1 . METHOD_3 ( command . METHOD_5 ( ) ) ; }
################################
public void METHOD_1 ( org.ovirt.engine.core.compat.Guid VAR_1 ) { VAR_2 . METHOD_1 ( VAR_1 ) ; }
############     ###############
public void METHOD_1 ( org.ovirt.engine.core.compat.Guid VAR_1 ) { this . VAR_1 = VAR_1 ; }
################################
public void METHOD_1 ( TYPE_1 < ? extends TYPE_2 > VAR_1 , java.lang.Object sender , TYPE_2 args ) { if ( ( METHOD_2 ( ) . getEntity ( ) ) == true ) { METHOD_3 ( ) . setEntity ( false ) ; } }
############     ###############
public void METHOD_1 ( TYPE_1 < ? extends TYPE_2 > VAR_1 , java.lang.Object sender , TYPE_2 args ) { if ( METHOD_2 ( ) . getEntity ( ) ) { METHOD_3 ( ) . setEntity ( false ) ; } }
################################
public TYPE_1 METHOD_1 ( java.lang.String id ) { return inject ( new

private void METHOD_1 ( ) { if ( ! ( VAR_1 ) ) { TYPE_1 . METHOD_2 ( VAR_2 . METHOD_3 ( VAR_3 ) ) . METHOD_4 ( TYPE_2 . METHOD_5 ( STRING_1 ) ) . METHOD_6 ( ) ; VAR_1 = true ; } }
############     ###############
private void METHOD_1 ( ) { if ( ! ( VAR_1 ) ) { org.mockito.Mockito.when ( VAR_2 . METHOD_3 ( VAR_3 ) ) . thenReturn ( TYPE_2 . METHOD_5 ( STRING_1 ) ) ; VAR_1 = true ; } }
################################
private org.ovirt.engine.core.common.businessentities . VdsNetworkInterface METHOD_1 ( java.lang.String VAR_1 ) { org.ovirt.engine.core.common.businessentities . VdsNetworkInterface VAR_2 = new org.ovirt.engine.core.common.businessentities . VdsNetworkInterface ( ) ; VAR_2 . METHOD_2 ( VAR_1 ) ; VAR_2 . METHOD_3 ( VAR_3 ) ; return VAR_2 ; }
############     ###############
private org.ovirt.engine.core.common.businessentities.network.VdsNetworkInterface METHOD_1 ( java.lang.String VAR_1 ) { org.ovirt.engine.core.common.businessentities.network.VdsNetworkInterface VAR_2 = ne

TYPE_1 . VdcQueryReturnValue runQuery ( TYPE_1 . VdcQueryType VAR_1 , TYPE_2 parameters ) ;
################################
private java.lang.String METHOD_1 ( TYPE_1 . Version version ) { return version == null ? null : TYPE_2 . METHOD_2 ( VAR_1 , version . METHOD_3 ( ) , version . METHOD_4 ( ) ) ; }
############     ###############
static java.lang.String METHOD_1 ( TYPE_1 . Version version ) { return version == null ? null : TYPE_2 . METHOD_2 ( VAR_1 , version . METHOD_3 ( ) , version . METHOD_4 ( ) ) ; }
################################
public static TYPE_1 METHOD_1 ( java.util.Map < java.lang.String , java.lang.Object > VAR_1 ) { TYPE_1 VAR_2 = new TYPE_1 ( ) ; TYPE_2 . METHOD_2 ( VAR_2 , VAR_1 ) ; return VAR_2 ; }
############     ###############
public static TYPE_1 METHOD_1 ( java.util.Map < java.lang.String , java.lang.Object > VAR_1 , org.ovirt.engine.core.common.businessentities.VDS host ) { TYPE_1 VAR_2 = new TYPE_1 ( ) ; TYPE_2 . METHOD_2 ( VAR_2 , VAR_1 , host ) ; return

In [186]:
file_fixed.close()
file_buggy.close()

In [188]:
import sys

def main(buggy, fixed, prediction):
    
    count_perfect = 0
    count_changed = 0
    count_bad = 0

    file_buggy = open(buggy, 'r')
    file_fixed = open(fixed, 'r')
    file_prediction = open(prediction, 'r')
    
    for line in file_buggy:
        buggy_line = line.strip()
        fixed_line = file_fixed.readline().strip()
        prediction_line = file_prediction.readline().strip()
        
        if(fixed_line == prediction_line):
            print(buggy_line)
            print("############     ###############")
            print(fixed_line)
            print("################################")
            count_perfect += 1
        
        elif(buggy_line == prediction_line):
            count_bad += 1

        else:
            count_changed += 1

    file_buggy.close()
    file_fixed.close()
    file_prediction.close()
    print((str(count_perfect) + " " + str(count_changed) + " " + str(count_bad)))

main("tufano_train_data/buggy.txt","tufano_train_data/fixed.txt", "tufano_train_data/predictions.txt")


public int METHOD_1 ( android.bluetooth.BluetoothDevice device ) { if ( VAR_1 ) android.util.Log.v ( TYPE_1 . TAG , STRING_1 ) ; TYPE_1 service = METHOD_2 ( ) ; if ( service == null ) return VAR_2 ; return service . METHOD_1 ( device ) ; }
############     ###############
public int METHOD_1 ( android.bluetooth.BluetoothDevice device ) { if ( VAR_1 ) { android.util.Log.v ( TYPE_1 . TAG , STRING_1 ) ; } TYPE_1 service = METHOD_2 ( ) ; if ( service == null ) { return VAR_2 ; } return service . METHOD_1 ( device ) ; }
################################
public long METHOD_1 ( int i ) { if ( ! ( VAR_1 ) ) { throw new TYPE_1 ( STRING_1 ) ; } return METHOD_1 ( METHOD_2 ( METHOD_3 ( i , VAR_2 ) ) ) ; }
############     ###############
public final long METHOD_1 ( int i ) { if ( ! ( VAR_1 ) ) { throw new TYPE_1 ( STRING_1 ) ; } return METHOD_1 ( METHOD_2 ( METHOD_3 ( i , VAR_2 ) ) ) ; }
################################
synchronized TYPE_1 METHOD_1 ( int VAR_1 ) { METHOD_2 ( ) ; return METHOD_3 ( 

public void METHOD_1 ( ) { try { TYPE_1 . METHOD_2 ( STRING_1 ) ; fail ( ) ; } catch ( java.lang.IllegalArgumentException expected ) { } }
################################
native void METHOD_1 ( int VAR_1 , int VAR_2 ) ;
############     ###############
native void METHOD_1 ( long VAR_1 , long VAR_2 ) ;
################################
public TYPE_1 METHOD_1 ( int VAR_1 , java.lang.String VAR_2 ) throws android.os.RemoteException { synchronized ( VAR_3 ) { METHOD_2 ( ) ; return METHOD_3 ( VAR_4 ) . METHOD_1 ( VAR_5 , VAR_6 , VAR_1 , VAR_2 ) ; } }
############     ###############
public TYPE_1 METHOD_1 ( int VAR_1 , java.lang.String VAR_2 ) throws android.os.RemoteException { synchronized ( VAR_3 ) { METHOD_2 ( ) ; return METHOD_3 ( VAR_4 ) . METHOD_1 ( VAR_1 , VAR_2 ) ; } }
################################
public static void METHOD_1 ( java.lang.String [ ] args ) throws java.lang.Exception { TYPE_1 . METHOD_2 ( args [ 1 ] ) ; Main . METHOD_3 ( ) ; }
############     ###############
pub

In [ ]:
1 second bracket 
2. int ->long ......
    public-private
    int->float
3. add final ..
4. deleting code part(like delete this)  ......
5. try transfer .
6. void-abstract and delete code part
7. adding or deleting a parameter in the last of function


In [191]:
import sys

def main(buggy, fixed, prediction):
    ans = 0
    count_perfect = 0
    count_changed = 0
    count_bad = 0

    file_buggy = open(buggy, 'r')
    file_fixed = open(fixed, 'r')
    file_prediction = open(prediction, 'r')
    
    for line in file_buggy:
        buggy_line = line.strip()
        fixed_line = file_fixed.readline().strip()
        prediction_line = file_prediction.readline().strip()
        if(fixed_line == prediction_line):
            if(len(fixed_line)<=len(buggy_line)):
                ans+=1
            count_perfect += 1
        
        elif(buggy_line == prediction_line):
            count_bad += 1

        else:
            count_changed += 1

    file_buggy.close()
    file_fixed.close()
    file_prediction.close()
    print(ans)
    print(count_perfect)
    #print((str(count_perfect) + " " + str(count_changed) + " " + str(count_bad)))

main("tufano_train_data/buggy.txt","tufano_train_data/fixed.txt", "tufano_train_data/predictions.txt")


51
78


# test data analysis

In [197]:
import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
def read(path):
    return open(path, 'r', encoding = 'utf-8').read()


BASE="C:/research_stuff/codes/"
test_data = getJsonData('test_by_project.json')
print(len(test_data))

3407


In [198]:
test_data[0].keys()

dict_keys(['status', 'message', 'comment_id', 'target', 'code_snippet', 'prime_var_dic', 'class_list', 'func_list', 'tokenized_code_snippet', 'tokenized_target', 'tokenized_comment', 'global_index', 'base_code_line_number', 'base_patch_number', 'changed_patch_number', 'code_file_name', 'line_change', 'written_on', 'project_name', 'int_date'])

In [424]:
idx+=1
print(test_data[idx]['global_index'])
print("########################")
print(test_data[idx]['message'])
print("########################")
print(test_data[idx]['code_snippet'])
print("########################")
print(test_data[idx]['target'])
print("########################")


2860
########################
rename?
########################
<|startfocus|>
    private boolean isInvalidSlotId(int slotIdx) {
        if (slotIdx < 0 || slotIdx >= mSlotIdxToSubId.length) return true;
<|endfocus|>
        return false;
########################
    private boolean isInvalidSlotId(int slotIndex) {
        if (slotIndex < 0 || slotIndex >= mSlotIndexToSubId.length) return true;

########################


In [ ]:
variable_value_change = 5
bug_fix = 30    (2807, 122 compilation error)
Assert_checkpoint_adding = 4
optimization = 33 (2783, 120), return early (import optimization 2820,138) (code optimization by optimizing imports)
code_style_change = 7 (2465, 159)
del_optimization = 4
test_successful_del = 1
method_name_change = 9
variable_name_change = 23
class_name_change = 2
string_change = 1
indentation = 4
wrap_for_code_readibiliuty = 5 (2303, 174)
typo = 5 (2473, 160)

log_change = 7
documentation = 16
comment = 19
copyright_change = 1 

not_understood_index = (2382, 1), (2621, 17) (2640, 18) (2661, 44) (1521, 69) (2671, 107) (1536, 125) (2650, 130) (2805, 134)
                    (2806, 135) (2826,143) (2044, 145) (2812, 152)
ditto
false data (2872,158)
example 
(2814,154 )

In [425]:
idx

195